# Covertype Data Set Preprocessing

In [1]:
import csv
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

## Preparing the datasets for KFP and TFX workshops

### Set the paths

In [2]:
FULL_DATASET = '../covertype.csv'
SMALL_DATASET= '../covertype_small.csv'

ORIGINAL_DATASET_PATH = 'gs://workshop-datasets/covertype/orig/covtype.data'

## Preprocess the original dataset

### Load the original dataset

In [3]:
df = pd.read_csv(ORIGINAL_DATASET_PATH, header=None)
print(df.shape)
df.head()

(581012, 55)


,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


### Configure soil type and wilderness area domains

In [4]:
soil_type = [
"1", "C2702", "Cathedral family - Rock outcrop complex, extremely stony.",
"2", "C2703", "Vanet - Ratake families complex, very stony.",
"3", "C2704", "Haploborolis - Rock outcrop complex, rubbly.",
"4", "C2705", "Ratake family - Rock outcrop complex, rubbly.",
"5", "C2706", "Vanet family - Rock outcrop complex complex, rubbly.",
"6", "C2717", "Vanet - Wetmore families - Rock outcrop complex, stony.",
"7", "C3501", "Gothic family.",
"8", "C3502", "Supervisor - Limber families complex.",
"9", "C4201", "Troutville family, very stony.",
"10", "C4703", "Bullwark - Catamount families - Rock outcrop complex, rubbly.",
"11", "C4704", "Bullwark - Catamount families - Rock land complex, rubbly.",
"12", "C4744", "Legault family - Rock land complex, stony.",
"13", "C4758", "Catamount family - Rock land - Bullwark family complex, rubbly.",
"14", "C5101", "Pachic Argiborolis - Aquolis complex.",
"15", "C5151", "unspecified in the USFS Soil and ELU Survey.",
"16", "C6101", "Cryaquolis - Cryoborolis complex.",
"17", "C6102", "Gateview family - Cryaquolis complex.",
"18", "C6731", "Rogert family, very stony.",
"19", "C7101", "Typic Cryaquolis - Borohemists complex.",
"20", "C7102", "Typic Cryaquepts - Typic Cryaquolls complex.",
"21", "C7103", "Typic Cryaquolls - Leighcan family, till substratum complex.",
"22", "C7201", "Leighcan family, till substratum, extremely bouldery.",
"23", "C7202", "Leighcan family, till substratum - Typic Cryaquolls complex.",
"24", "C7700", "Leighcan family, extremely stony.",
"25", "C7701", "Leighcan family, warm, extremely stony.",
"26", "C7702", "Granile - Catamount families complex, very stony.",
"27", "C7709", "Leighcan family, warm - Rock outcrop complex, extremely stony.",
"28", "C7710", "Leighcan family - Rock outcrop complex, extremely stony.",
"29", "C7745", "Como - Legault families complex, extremely stony.",
"30", "C7746", "Como family - Rock land - Legault family complex, extremely stony.",
"31", "C7755", "Leighcan - Catamount families complex, extremely stony.",
"32", "C7756", "Catamount family - Rock outcrop - Leighcan family complex, extremely stony.",
"33", "C7757", "Leighcan - Catamount families - Rock outcrop complex, extremely stony.",
"34", "C7790", "Cryorthents - Rock land complex, extremely stony.",
"35", "C8703", "Cryumbrepts - Rock outcrop - Cryaquepts complex.",
"36", "C8707", "Bross family - Rock land - Cryumbrepts complex, extremely stony.",
"37", "C8708", "Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.",
"38", "C8771", "Leighcan - Moran families - Cryaquolls complex, extremely stony.",
"39", "C8772", "Moran family - Cryorthents - Leighcan family complex, extremely stony.",
"40", "C8776", "Moran family - Cryorthents - Rock land complex, extremely stony.",
]

wilderness_area = [
"Rawah", "Rawah Wilderness Area",
"Neota", "Neota Wilderness Area",
"Commanche", "Comanche Peak Wilderness Area",
"Cache", "Cache la Poudre Wilderness Area"
]

### Map one-hot encoded values to categorical domains

In [5]:
soil = df.loc[:, 14:53].apply(lambda x: soil_type[1::3][x.to_numpy().nonzero()[0][0]], axis=1)
soil

0         C7745
1         C7745
2         C4744
3         C7746
4         C7745
          ...  
581007    C2703
581008    C2703
581009    C2703
581010    C2703
581011    C2703
Length: 581012, dtype: object

In [6]:
wilderness = df.loc[:, 10:13].apply(lambda x: wilderness_area[0::2][x.to_numpy().nonzero()[0][0]], axis=1)
wilderness

0             Rawah
1             Rawah
2             Rawah
3             Rawah
4             Rawah
            ...    
581007    Commanche
581008    Commanche
581009    Commanche
581010    Commanche
581011    Commanche
Length: 581012, dtype: object

### Create a dataset with column names and categorical values replacing one-hot encoded soil type and wilderness areas

In [7]:
COLUMN_NAMES = [
    'Elevation', 
    'Aspect', 
    'Slope', 
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points',
    'Wilderness_Area',
    'Soil_Type',
    'Cover_Type']

df_full = pd.concat([df.loc[:, 0:9], wilderness, soil, df.loc[:, 54]], axis=1, ignore_index=True)
df_full.columns = COLUMN_NAMES
df_full

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,5
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,5
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,2
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,2
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,C2703,3
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,C2703,3
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,C2703,3
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,C2703,3


### Convert the label to 0-6 range

In [8]:
df_full['Cover_Type'] = df_full['Cover_Type'] - 1

### Save the dataset to CSV file

In [9]:
df_full.to_csv(FULL_DATASET, header=True, index=False)

In [10]:
!head $FULL_DATASET

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,4
2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,4
2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,1
2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,1
2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,4
2579,132,6,300,-15,67,230,237,140,6031,Rawah,C7745,1
2606,45,7,270,5,633,222,225,138,6256,Rawah,C7745,4
2605,49,4,234,7,573,222,230,144,6228,Rawah,C7745,4
2617,45,9,240,56,666,223,221,133,6244,Rawah,C7745,4


### Create a small version

In [11]:
df_small, df_other = train_test_split(df_full, train_size=100000, stratify=df_full.Cover_Type)

In [12]:
df_small.to_csv(SMALL_DATASET, header=True, index=False)

### Copy the datasets to GCS

In [13]:
!gsutil cp $FULL_DATASET gs://workshop-datasets/covertype/full/dataset.csv
!gsutil cp $SMALL_DATASET gs://workshop-datasets/covertype/small/dataset.csv

Copying file://../covertype.csv [Content-Type=text/csv]...
- [1 files][ 30.5 MiB/ 30.5 MiB]                                                
Operation completed over 1 objects/30.5 MiB.                                     
Copying file://../covertype_small.csv [Content-Type=text/csv]...
/ [1 files][  5.3 MiB/  5.3 MiB]                                                
Operation completed over 1 objects/5.3 MiB.                                      


## Preparing the datasets for the Data Analysis and Validation workshop

### Set the paths

In [14]:
TRAINING_DATASET='../covertype_training.csv'
TRAINING_DATASET_WITH_MISSING = '../covertype_training_missing.csv'
EVALUATION_DATASET='../covertype_evaluation.csv'
EVALUATION_DATASET_WITH_ANOMALIES='../covertype_evaluation_anomalies.csv'
SERVING_DATASET='../covertype_serving.csv'

### Convert Soil_Type to integer literal

In the datasets for this workshop  the `Soil_Type` field is encoded as the `Integer` literal.

In [15]:
df_full = df = pd.read_csv(FULL_DATASET, dtype={'Soil_Type': object})
df_full

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,4
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,4
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,1
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,1
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,C2703,2
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,C2703,2
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,C2703,2
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,C2703,2


In [16]:
df_full['Soil_Type'] = df_full['Soil_Type'].str.slice(start=1)
df_full

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,7745,4
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,7745,4
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,4744,1
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,7746,1
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,7745,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,2703,2
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,2703,2
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,2703,2
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,2703,2


###  Create a split without 5151 soil type

In [17]:
df_full.Soil_Type.value_counts()

7745    115247
7202     57752
7756     52519
7757     45154
7201     33373
4703     32634
7746     30170
4744     29971
7755     25666
7700     21278
4758     17431
8771     15573
8772     13806
4704     12410
2705     12396
7102      9259
8776      8750
2703      7525
2717      6575
2704      4823
7101      4021
6102      3422
2702      3031
6101      2845
7702      2589
6731      1899
8703      1891
7790      1611
2706      1597
4201      1147
7709      1086
7710       946
7103       838
5101       599
7701       474
8708       298
3502       179
8707       119
3501       105
5151         3
Name: Soil_Type, dtype: int64

In [18]:
df_5151 = df_full[df_full['Soil_Type']=='5151']
df_no_5151 = df_full[df_full['Soil_Type']!='5151']

In [19]:
df_5151

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
241543,2078,34,10,0,0,212,219,218,134,484,Cache,5151,5
241544,2080,13,19,30,0,192,198,197,132,499,Cache,5151,5
241545,2076,27,24,30,5,175,201,180,105,516,Cache,5151,5


In [20]:
df_no_5151

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,7745,4
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,7745,4
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,4744,1
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,7746,1
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,7745,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,2703,2
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,2703,2
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,2703,2
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,2703,2


### Create training, evaluation, and serving splits

In [21]:
df_train, df_other = train_test_split(df_no_5151, train_size=431009, stratify=df_no_5151.Cover_Type)
df_evaluate, df_serving = train_test_split(df_other, train_size=75000, stratify=df_other.Cover_Type)
df_serving = df_serving.drop(columns=['Cover_Type'])
print(df_train.shape)
print(df_evaluate.shape)
print(df_serving.shape)

(431009, 13)
(75000, 13)
(75000, 12)


Add some missing values to the training split.

In [22]:
df_train_missing = df_train.reset_index(drop=True)
df_train_missing.loc[0:8999, 'Horizontal_Distance_To_Hydrology'] = None
df_train_missing

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,3034,235,13,NaN,134,5799,197,251,190,4094,Rawah,7745,1
1,2827,26,16,NaN,13,3511,211,204,125,6008,Rawah,7745,1
2,2854,27,12,NaN,22,618,215,214,134,1295,Commanche,4704,1
3,3188,156,13,NaN,28,3666,235,241,134,1719,Commanche,7756,1
4,3091,61,8,NaN,10,1390,227,223,129,2868,Commanche,7202,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431004,3049,290,5,663.0,32,1380,206,239,172,787,Commanche,7101,1
431005,2776,43,13,60.0,-10,2737,222,212,122,2002,Commanche,7700,1
431006,2970,61,21,365.0,175,390,232,191,82,1377,Rawah,7745,0
431007,2975,7,10,272.0,76,3421,208,221,150,2056,Rawah,7745,1


Create the evaluation split where some values of Slope are more than 90 degrees and 3 examples have 5151 code for soil type, which is not present in the training split.

In [23]:
df_evaluate_anomalies = df_evaluate.reset_index(drop=True)
df_evaluate_anomalies.loc[0:4, 'Slope'] = 110
df_evaluate_anomalies = pd.concat([df_evaluate_anomalies, df_5151])
df_evaluate_anomalies

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2814,55,110,30,8,2483,228,201,100,2727,Rawah,7202,1
1,2903,149,110,108,7,5148,225,239,149,4713,Rawah,6101,1
2,3338,236,110,300,27,1008,202,249,184,1620,Commanche,7756,0
3,3305,153,110,60,9,2353,240,237,118,324,Commanche,8703,6
4,2788,107,110,0,0,2533,233,232,131,3896,Rawah,7202,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74998,3125,84,10,283,43,1980,234,224,121,3264,Commanche,7700,0
74999,3361,337,20,514,158,2285,172,208,173,920,Commanche,8772,6
241543,2078,34,10,0,0,212,219,218,134,484,Cache,5151,5
241544,2080,13,19,30,0,192,198,197,132,499,Cache,5151,5


In [24]:
df_evaluate_anomalies.Soil_Type.value_counts()

7745    14849
7202     7515
7756     6723
7757     5886
7201     4271
4703     4242
4744     3838
7746     3821
7755     3337
7700     2829
4758     2328
8771     1986
8772     1792
2705     1547
4704     1527
8776     1178
7102     1145
2703      989
2717      910
2704      582
7101      496
6102      476
2702      397
6101      354
7702      341
8703      238
6731      237
2706      206
7790      202
7709      146
4201      139
7710      128
7103      107
5101       77
7701       62
8708       37
3502       25
8707       20
3501       17
5151        3
Name: Soil_Type, dtype: int64

### Save the splits to local files.

In [25]:
df_train.to_csv(TRAINING_DATASET, header=True, index=False)
df_train_missing.to_csv(TRAINING_DATASET_WITH_MISSING, header=True, index=False)
df_evaluate.to_csv(EVALUATION_DATASET, header=True, index=False)
df_evaluate_anomalies.to_csv(EVALUATION_DATASET_WITH_ANOMALIES, header=True, index=False)
df_serving.to_csv(SERVING_DATASET, header=True, index=False)

### Copy the splits to GCS

In [26]:
!gsutil cp $TRAINING_DATASET gs://workshop-datasets/covertype/data_validation/training/dataset.csv
!gsutil cp $TRAINING_DATASET_WITH_MISSING gs://workshop-datasets/covertype/data_validation/training_missing/dataset.csv
!gsutil cp $EVALUATION_DATASET gs://workshop-datasets/covertype/data_validation/evaluation/dataset.csv
!gsutil cp $EVALUATION_DATASET_WITH_ANOMALIES gs://workshop-datasets/covertype/data_validation/evaluation_anomalies/dataset.csv
!gsutil cp $SERVING_DATASET gs://workshop-datasets/covertype/data_validation/serving/dataset.csv

Copying file://../covertype_training.csv [Content-Type=text/csv]...
- [1 files][ 22.2 MiB/ 22.2 MiB]                                                
Operation completed over 1 objects/22.2 MiB.                                     
Copying file://../covertype_training_missing.csv [Content-Type=text/csv]...
- [1 files][ 23.0 MiB/ 23.0 MiB]                                                
Operation completed over 1 objects/23.0 MiB.                                     
Copying file://../covertype_evaluation.csv [Content-Type=text/csv]...
/ [1 files][  3.9 MiB/  3.9 MiB]                                                
Operation completed over 1 objects/3.9 MiB.                                      
Copying file://../covertype_evaluation_anomalies.csv [Content-Type=text/csv]...
/ [1 files][  3.9 MiB/  3.9 MiB]                                                
Operation completed over 1 objects/3.9 MiB.                                      
Copying file://../covertype_serving.csv [Content-Type=